# Création des scripts d'activation pour environnements multiples
Ce notebook génère les scripts nécessaires pour activer l'environnement virtuel `.venv` depuis différents environnements (Git Bash, WSL, Linux

In [ ]:
# Créer le script d'activation universel
import os
import platform

# Détecter si nous sommes dans un environnement PyCharm
in_pycharm = 'PYCHARM_HOSTED' in os.environ

# Créer le répertoire scripts s'il n'existe pas
if not os.path.exists('scripts'):
    os.makedirs('scripts')

# Script d'activation pour bash (Linux, Git Bash, WSL)
bash_script = """#!/bin/bash

# Détection de l'environnement
if [[ "$OSTYPE" == "msys" ]]; then
    # Git Bash sous Windows
    ENV_TYPE="Git Bash"
    VENV_PATH="$(pwd)/.venv/Scripts/activate"
elif [[ -n "$WSL_DISTRO_NAME" ]]; then 
    # WSL (Windows Subsystem for Linux)
    ENV_TYPE="WSL"
    VENV_PATH="$(pwd)/.venv/bin/activate"
else
    # Linux natif ou macOS
    ENV_TYPE="Linux/macOS"
    VENV_PATH="$(pwd)/.venv/bin/activate"
fi

echo "Environnement détecté: $ENV_TYPE"
echo "Activation de l'environnement virtuel..."

if [ -f "$VENV_PATH" ]; then
    source "$VENV_PATH"
    echo "Environnement virtuel activé avec succès!"
    echo "Python: $(which python)"
    echo "Version: $(python --version)"
else
    echo "ERREUR: Environnement virtuel non trouvé à $VENV_PATH"
    echo "Assurez-vous d'exécuter ce script depuis la racine du projet."
    exit 1
fi
"""

# Script d'activation pour Windows CMD
cmd_script = """@echo off
REM Script d'activation pour Windows CMD

set VENV_PATH=%CD%\\.venv\\Scripts\\activate.bat

echo Environnement: Windows CMD
echo Activation de l'environnement virtuel...

if exist %VENV_PATH% (
    call %VENV_PATH%
    echo Environnement virtuel activé avec succès!
    where python
    python --version
) else (
    echo ERREUR: Environnement virtuel non trouvé à %VENV_PATH%
    echo Assurez-vous d'exécuter ce script depuis la racine du projet.
    exit /b 1
)
"""

# Script d'activation pour PowerShell
ps_script = """# Script d'activation pour PowerShell

$venvPath = Join-Path -Path $PWD -ChildPath ".venv\\Scripts\\Activate.ps1"

Write-Host "Environnement: PowerShell"
Write-Host "Activation de l'environnement virtuel..."

if (Test-Path $venvPath) {
    & $venvPath
    Write-Host "Environnement virtuel activé avec succès!"
    Get-Command python | Select-Object -ExpandProperty Source
    python --version
} else {
    Write-Host "ERREUR: Environnement virtuel non trouvé à $venvPath" -ForegroundColor Red
    Write-Host "Assurez-vous d'exécuter ce script depuis la racine du projet." -ForegroundColor Red
    exit 1
}
"""

# Écrire les scripts dans le répertoire scripts
with open('scripts/activate.sh', 'w', newline='\n') as f:
    f.write(bash_script)

with open('scripts/activate.bat', 'w', newline='\r\n') as f:
    f.write(cmd_script)

with open('scripts/activate.ps1', 'w', newline='\n') as f:
    f.write(ps_script)

# Rendre le script bash exécutable si on est sous Linux/MacOS
if platform.system() != "Windows":
    os.chmod('scripts/activate.sh', 0o755)

print("Scripts d'activation créés avec succès dans le dossier 'scripts/'")

In [ ]:
# Création du fichier .gitignore adapté pour .venv
gitignore_content = """# .venv - on ignore par défaut, mais on garde certains éléments
.venv/*

# Ne pas ignorer les scripts d'activation (pour faciliter le passage entre environnements)
!.venv/Scripts/activate*
!.venv/Scripts/python.exe
!.venv/Scripts/pip.exe
!.venv/bin/activate*
!.venv/bin/python
!.venv/bin/pip

# Ignorer les fichiers compilés Python
__pycache__/
*.py[cod]
*$py.class

# Ignorer les fichiers de distribution
dist/
build/
*.egg-info/

# Ignorer les fichiers d'environnement
.env
"""

# Créer ou mettre à jour le fichier .gitignore
with open('.gitignore', 'w', newline='\n') as f:
    f.write(gitignore_content)

print("Fichier .gitignore créé/mis à jour pour gérer correctement .venv")